In [1]:
# stdlib
from unittest import mock

# third party
import hagrid
import numpy as np

# syft absolute
import syft as sy
from syft.client.domain_client import DomainClient
from syft.client.gateway_client import GatewayClient

print(f"{hagrid.__version__ = }")
print(f"{sy.__version__ = }")

hagrid.__version__ = '0.3.112'
sy.__version__ = '0.8.6-beta.1'


#### Deleting existing nodes
```bash
bash -c '\
    CLUSTER_NAME=testgateway1 tox -e dev.k8s.destroy || true && \
    CLUSTER_NAME=testdomain1 tox -e dev.k8s.destroy || true && \
    CLUSTER_NAME=testdomain2 tox -e dev.k8s.destroy || true'
```


#### Launch a Gateway Node

```bash
bash -c '\
    export CLUSTER_NAME=testgateway1 CLUSTER_HTTP_PORT=9081 DEVSPACE_PROFILE=gateway && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

#### Launch 2 Domain Nodes
```bash
bash -c '\
    export CLUSTER_NAME=testdomain1 CLUSTER_HTTP_PORT=9091 && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

```bash
bash -c '\
    export CLUSTER_NAME=testdomain2 CLUSTER_HTTP_PORT=9092 && \
    tox -e dev.k8s.start && \
    tox -e dev.k8s.hotreload'
```

In [2]:
!k3d cluster list

NAME           SERVERS   AGENTS   LOADBALANCER
testgateway1   1/1       0/0      true


In [14]:
def mock_load_network_registry_json(network_host: str, network_port: int) -> dict:
    return {
        "2.0.0": {
            "gateways": [
                {
                    "name": "test-gateway",
                    "host_or_ip": f"{network_host}",
                    "protocol": "http",
                    "port": f"{network_port}",
                    "admin_email": "support@openmined.org",
                    "website": "https://www.openmined.org/",
                    "slack": "https://slack.openmined.org/",
                    "slack_channel": "#support",
                }
            ]
        }
    }


with mock.patch(
    "syft.client.registry.NetworkRegistry.load_network_registry_json",
    return_value=mock_load_network_registry_json("localhost", 9081),
):
    print(sy.gateways)
    print(sy.domains)
    print(sy.search(name="dataset_1"))
    a = sy.search(name="dataset_1")
    print(a[0].name)

           name host_or_ip protocol  port            admin_email                     website                         slack slack_channel        version
0  test-gateway  localhost     http  9081  support@openmined.org  https://www.openmined.org/  https://slack.openmined.org/      #support  0.8.5-beta.10
     name organization        version host_or_ip protocol  port                                id
0  mynode    OpenMined  0.8.5-beta.10  localhost     http  9082  b213ef29c67f444c8aa5e8603beff97b
[syft.service.dataset.dataset.Dataset]
dataset_1


In [4]:
gateway_client: GatewayClient = sy.login(
    port=9081, email="info@openmined.org", password="changethis"
)

Logged into <mynode: High side Gateway> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
gateway_client: GatewayClient = sy.login(
    port=9081, email="info@openmined.org", password="changethis"
)
domain_client1: DomainClient = sy.login(
    port=9082, email="info@openmined.org", password="changethis"
)
domain_client3: DomainClient = sy.login(
    port=9083, email="info@openmined.org", password="changethis"
)

ConnectionError: Failed to fetch http://localhost:9081/api/v2/metadata. Response returned with code 502

In [4]:
domain_client1: DomainClient = sy.login(
    port=9082, email="info@openmined.org", password="changethis"
)

ConnectionError: Failed to fetch http://localhost:9082/api/v2/metadata. Response returned with code 404

In [6]:
res = domain_client1.connect_to_gateway(gateway_client)

In [7]:
res

SyftSuccess: Connected domain to mynode gateway

In [9]:
input_data = np.array([1, 2, 3])
mock_data = np.array([4, 5, 6])

asset_name = "asset_1"
asset = sy.Asset(name=asset_name, data=input_data, mock=mock_data)
dataset_name = "dataset_1"
dataset = sy.Dataset(name=dataset_name, asset_list=[asset])

In [10]:
dataset_res = domain_client1.upload_dataset(dataset)

  0%|                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: asset_1



100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.42it/s]


In [11]:
dataset_res

SyftSuccess: Dataset uploaded to 'mynode'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`